# Visualising GeoGraph Interactively

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ai4er-cdt/gtc-biodiversity/main?filepath=notebooks%2F3-demo-geographviewer-polesia.ipynb)

This tutorial shows how to visualise a GeoGraph on an interactive map.

---

## 1. Setup and Loading package

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [1]:
import ipyleaflet
import geograph
from geograph.visualisation import geoviewer
from geograph.constants import UTM35N
from geograph.demo.binder_constants import DATA_DIR

---

## 2. Loading Data

---

## 3. Creating `GeoGraph`

In [3]:
# Building the main graph structure
graph = geograph.GeoGraph(
    gdf, crs=UTM35N, columns_to_rename={"Eunis_name": "class_label", "AREA": "area"}
)

Step 1 of 2: Creating nodes and finding neighbours: 100%|██████████| 323/323 [00:00<00:00, 569.78it/s]
Step 2 of 2: Adding edges: 100%|██████████| 323/323 [00:00<00:00, 66710.67it/s]

Graph successfully loaded with 323 nodes and 816 edges.


---

## 4. Creating Habitats

In [4]:
# First selecting the classes that make up our habitat
# We chose all classes with 'pine' in the name.
pine_classes = [label for label in graph.df.class_label.unique() if "pine" in label]
pine_classes

['Subcontinental moss Scots pine forests',
 'Subcontinental lichen Scots pine forests',
 'Subcontinental moorgrass Scots pine forests',
 'Boreal Labrador tea Scots pine bog woods',
 'Boreal cottonsedge Scots pine bog woods',
 'Boreal neutrocline sphagnum Scots pine fen woods',
 'Mixed Scots pine-birch woodland']

In [5]:
# Distances: mobile (<100m), semi mobile (<25m) and sessile (<5m)
# (proposed by Adham Ashton-Butt at BTO)
graph.add_habitat("Sessile", max_travel_distance=5, valid_classes=pine_classes)

graph.add_habitat("Semi mobile", max_travel_distance=25, valid_classes=pine_classes)

graph.add_habitat("Mobile", max_travel_distance=500, valid_classes=pine_classes)

Constructing graph: 100%|██████████| 39/39 [00:00<00:00, 11990.75it/s]


Calculating components...
Habitat successfully loaded with 95 nodes and 78 edges.


Constructing graph: 100%|██████████| 36/36 [00:00<00:00, 12607.08it/s]


Calculating components...
Habitat successfully loaded with 95 nodes and 86 edges.


Constructing graph: 100%|██████████| 14/14 [00:00<00:00, 7885.09it/s]

Calculating components...
Habitat successfully loaded with 95 nodes and 214 edges.


---

## 5. Interactive Graph

In [6]:
viewer = geoviewer.GeoGraphViewer(small_screen=True)
viewer.add_layer(ipyleaflet.basemaps.Esri.WorldImagery)
viewer.add_graph(graph, name="Polesia data", with_components=True)
viewer.enable_graph_controls()
viewer

Constructing graph: 100%|██████████| 14/14 [00:00<00:00, 7224.44it/s]


GeoGraphViewer(center=[51.389167, 30.099444], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

> Note: an interactive viewer will show up here.